In [ ]:
import pandas as pd
import googlemaps

gmaps = googlemaps.Client(key='<API-KEY>')

In [35]:
# obtain name, address, rating, website, open hour, and cuisine type of 4 food trucks
trucks = gmaps.places(query="Food truck", location="39.791000,-86.148003")['results']
len(trucks)

20

In [41]:
place_ids = [place['place_id'] for place in trucks[0:6]]
def format_time(time):
  formatted = time[0:2] + ":" + time[2:]
  if int(time) > 1200:
    formatted += " PM"
  else:
    formatted += " AM"
  return formatted

places = []
for id in place_ids:
  place = gmaps.place(id)['result']
  keys = ["name", "formatted_address", "rating", "website", "current_opening_hours", "types"]
  hasAllKeys = True
  for key in keys:
    if not key in place.keys():
      hasAllKeys = False
  if not hasAllKeys:
    continue
  places.append({
      "name": place["name"],
      "address": place["formatted_address"],
      "rating": place["rating"],
      "website": place["website"],
      "open_hour": format_time(place["current_opening_hours"]["periods"][0]["open"]["time"]),
      "cuisine_type": place['types'][0]
  })
places

[{'name': 'ClusterTruck',
  'address': '729 N Pennsylvania St, Indianapolis, IN 46204, USA',
  'rating': 4.8,
  'website': 'https://downtown-indy.clustertruck.com/menu?utm_source=google%20my%20business&utm_medium=search%20listing',
  'open_hour': '10:00 AM',
  'cuisine_type': 'meal_delivery'},
 {'name': 'Tha Corn Man',
  'address': '2442 Central Ave, Indianapolis, IN 46205, USA',
  'rating': 3.8,
  'website': 'https://tha-corn-man.ueniweb.com/?utm_campaign=gmb',
  'open_hour': '15:00 PM',
  'cuisine_type': 'meal_takeaway'},
 {'name': 'The Grub House',
  'address': '1953 N College Ave, Indianapolis, IN 46202, USA',
  'rating': 4.1,
  'website': 'http://grubhouseindy.com/',
  'open_hour': '08:00 AM',
  'cuisine_type': 'cafe'},
 {'name': 'The Night Owl Food Truck',
  'address': 'South and, S Meridian St, Indianapolis, IN 46225, USA',
  'rating': 5,
  'website': 'http://mercersnightowl.com/',
  'open_hour': '18:00 PM',
  'cuisine_type': 'restaurant'},
 {'name': 'Pico de Gallo Food Truck',


In [92]:
# Remove one food truck frrom list to make for 4; 2 with early opening, and 2 with late opening
filtered = places[0:1] + places[2:]
df = pd.DataFrame.from_dict(filtered)
df.to_csv("food_trucks.csv", index=False)
df

,name,address,rating,website,open_hour,cuisine_type
0,ClusterTruck,"729 N Pennsylvania St, Indianapolis, IN 46204,...",4.8,https://downtown-indy.clustertruck.com/menu?ut...,10:00 AM,meal_delivery
1,The Grub House,"1953 N College Ave, Indianapolis, IN 46202, USA",4.1,http://grubhouseindy.com/,08:00 AM,cafe
2,The Night Owl Food Truck,"South and, S Meridian St, Indianapolis, IN 462...",5.0,http://mercersnightowl.com/,18:00 PM,restaurant
3,Pico de Gallo Food Truck,"3014 Lafayette Rd #3000, Indianapolis, IN 4622...",4.5,https://www.facebook.com/peregrino1982/,17:00 PM,restaurant


In [91]:
from datetime import datetime
day1 = [filtered[0], filtered[2]]
day2 = [filtered[1], filtered[3]]
#time, name, address, cuisine, travel distance, travel time, transportation type
route_data = []
def get_route_data(dest, idx):

      dt = datetime.strptime(f"{21 + idx}/10/23 {dest['open_hour'][0:5]}", "%d/%m/%y %H:%M")
      directions = gmaps.directions("39.791000,-86.148003",
                          dest['address'],
                          mode="driving",
                          departure_time=dt)
      legs = directions[0]['legs']

      time_unit = legs[0]['duration']['text'].split(" ")[1]
      total_dist = 0
      total_time = 0
      for leg in legs:
        dist_unit = leg['distance']['text'].split(" ")[1]
        dist = float(leg['distance']['text'].split(' ')[0])
        if (dist_unit == 'ft'):
          dist /= 5280
        total_dist += dist
        total_time += float(leg['duration']['text'].split(' ')[0])
      dist_str = str(total_dist) + " " + dist_unit
      time_str = str(total_time) + " " + time_unit

      transportation_type = "driving"
      return {
          "time": dest["open_hour"],
          "name": dest["name"],
          "address": dest["address"],
          "cuisine_type": dest["cuisine_type"],
          "travel_distance": dist_str,
          "travel_time": time_str,
          "transportation_type": transportation_type
      }

day_1_data = pd.DataFrame.from_dict([get_route_data(day1[0], 0), get_route_data(day1[1], 0)])
day_1_data.to_csv("day_1.csv", index=False)
day_2_data = pd.DataFrame.from_dict([get_route_data(day2[0], 1), get_route_data(day2[1], 1)])
day_2_data.to_csv("day_2.csv", index=False)

In [89]:
day_1_data

,time,name,address,cuisine_type,travel_distance,travel_time,transportation_type
0,10:00 AM,ClusterTruck,"729 N Pennsylvania St, Indianapolis, IN 46204,...",meal_delivery,1.3 mi,5.0 mins,driving
1,18:00 PM,The Night Owl Food Truck,"South and, S Meridian St, Indianapolis, IN 462...",restaurant,4.0 mi,14.0 mins,driving


In [90]:
day_2_data

,time,name,address,cuisine_type,travel_distance,travel_time,transportation_type
0,08:00 AM,The Grub House,"1953 N College Ave, Indianapolis, IN 46202, USA",cafe,0.8 mi,3.0 mins,driving
1,17:00 PM,Pico de Gallo Food Truck,"3014 Lafayette Rd #3000, Indianapolis, IN 4622...",restaurant,4.8 mi,15.0 mins,driving


In [ ]:
df['rating'].mean()